In [203]:
import pandas as pd
import numpy as np

# INPUT
output_url = '../output/new.xlsx'
form_url = '../data/format.xlsx'
sheet_name = 'FORM'
header_end_row = 9  # Row To: ASIA LCD SERVICE
footer_starter_row = 20  # Row To: ASIA LCD SERVICE

data_url = '../data/ORIGIN.xlsx'
data_sheetname = 'Service-T3'
data_header_starter = 13

cd_col = 'CD no'
invoice_col = 'Invoice no'
bill_col = 'BILL'
declaration_col = 'Date of declaration'

# GET FORM
_format = pd.read_excel(form_url, sheet_name=sheet_name, header=None)
header = _format.iloc[:header_end_row]
footer = _format.iloc[(footer_starter_row - 1):]

data = pd.read_excel(data_url, sheet_name=data_sheetname, header=(data_header_starter-1), converters={cd_col: str, invoice_col: str})
data = data.loc[~pd.isna(data[invoice_col]) & ~pd.isna(data[bill_col])]
data[declaration_col] = data[declaration_col].ffill()
data[bill_col] = data[bill_col].ffill()
data[cd_col] = data[cd_col].ffill()
data[invoice_col] = data[invoice_col].ffill()

grouped = data.groupby(bill_col, sort=False)

writer = pd.ExcelWriter(output_url, engine='xlsxwriter')
wookbook = writer.book

# STYLE CONFIG
bold = wookbook.add_format({
    'bold': 1,
    'align': 'center',
})
table = wookbook.add_format({
    'border': 1,
})
center = wookbook.add_format(
    {'bold': True,
    'border': 1,
    'align': 'center', 
    'valign': 'vcenter'}
)
num_format = wookbook.add_format({'num_format': '#,##0.00'})

evacuate = None

for name, group in grouped:
    current_row = 0
    header.to_excel(writer, sheet_name=name, index=False, header=False, startrow=current_row)
    worksheet = writer.sheets[name]
    current_row += header_end_row

    # ------------ HBL and INVOICES -------------
    invoices = group['Invoice no'].unique()
    data_len = len(invoices)

    title = [np.nan] * data_len
    hbls = [np.nan] * data_len
    blank = [np.nan] * data_len
    another_title = [np.nan] * data_len

    hbls[0] = name
    another_title[0] = 'Invoice No.:'
    title[0] = 'HBL'

    data_to_fill = {
        'title': title,                          # Title doesn't matter
        'hbls':hbls,
        'blank':blank,
        'another_title': another_title,
        'invoices': invoices
    }

    data_to_fill = pd.DataFrame(data_to_fill)
    data_to_fill.to_excel(writer, sheet_name=name, index=False, header=False, startrow=(current_row + 1), startcol=1)

    current_row += data_to_fill.shape[0] + 1

    # ------------ CDS ---------------
    cds = group['CD no'].unique()
    data_len = len(cds)

    title = [np.nan] * data_len
    another_title = [np.nan] * data_len

    title[0] = 'CDS:'
    another_title[0] = 'POL/POD:'

    data_to_fill = {
        'title': title,                          # Title doesn't matter
        'cds':cds,
        'blank':[np.nan] * data_len,
        'another_title': another_title,
        'scribebe': [np.nan] * data_len
    }
    data_to_fill = pd.DataFrame(data_to_fill)
    data_to_fill.to_excel(writer, sheet_name=name, index=False, header=False, startrow=(current_row), startcol=1)

    current_row += data_to_fill.shape[0]

    # ------------ TERM AND TRUCK -------------
    data_to_fill = [['TERM:', np.nan, np.nan, 'TRUCK']]
    data_to_fill = pd.DataFrame(data_to_fill)
    data_to_fill.to_excel(writer, sheet_name=name, index=False, header=False, startrow=(current_row), startcol=1)

    current_row += data_to_fill.shape[0]

    # ------------ DATE DECLARATION -------------
    declaration_dates = group[declaration_col].unique()
    declaration_dates = [dt for dt in declaration_dates if not pd.isna(dt)]
    declaration_dates = [pd.to_datetime(str(x)).strftime('%d-%m-%Y') for x in declaration_dates]
    data_len = len(declaration_dates)

    title = [np.nan] * data_len
    another_title = [np.nan] * data_len

    title[0] = 'Date of declaration'
    another_title[0] = 'GWT (kg):'

    data_to_fill = {
        'title': title,                          # Title doesn't matter
        'dates': declaration_dates,
        'blank':[np.nan] * data_len,
        'another_title': another_title,
        'scribebe': [np.nan] * data_len
    }
    data_to_fill = pd.DataFrame(data_to_fill)
    data_to_fill.to_excel(writer, sheet_name=name, index=False, header=False, startrow=(current_row), startcol=1)

    current_row += data_to_fill.shape[0] + 1

    # --- SERVICE COST ----
    data_to_fill = [
        ['Service cost (VND)'],
    ]

    data_to_fill = pd.DataFrame(data_to_fill)
    data_to_fill.to_excel(writer, sheet_name=name, index=False, header=False, startrow=(current_row), startcol=1)

    current_row += data_to_fill.shape[0]

    worksheet.write(f'B{str(current_row + 1)}', 'No.', center)
    worksheet.write(f'C{str(current_row + 1)}', 'Freight', center)
    worksheet.write(f'D{str(current_row + 1)}', 'Rate', center)
    worksheet.write(f'E{str(current_row + 1)}', 'VAT', center)
    worksheet.write(f'F{str(current_row + 1)}', 'Pay on behalf', center)
    worksheet.write(f'G{str(current_row + 1)}', 'Remark', center)
 
    current_row += 1

    start_table = str(current_row + 1)
    k = 1
    total = 0
    data_to_fill = []
    for _, row in group.iterrows():
        data_to_fill.append([k, 'Customs Clearance', row['Custom Clearance'], row['vat'], row['Pay on behalf'], 'VAT 8%'])
        k += 1
        if not pd.isna(row['Pay on behalf']):
            total += row['Pay on behalf']
    data_to_fill.append([np.nan, 'TOTAL AMOUNT (VND)', np.nan, np.nan, total, np.nan])
    data_to_fill = pd.DataFrame(data_to_fill)

    prepare_jump = data_to_fill.shape[0]

    data_to_fill.to_excel(writer, sheet_name=name, index=False, header=False, startrow=(current_row), startcol=1)



    current_row += prepare_jump
    end_table = str(current_row)

    # WRITE FOOTER
    footer.to_excel(writer, name, index=False, header=False, startrow=current_row + 1)

    worksheet.conditional_format('A1:A2', 
        {
            'type': 'no_blanks',
            'format':   bold
        }
    )

    worksheet.conditional_format('B6:B7', 
        {
            'type': 'no_blanks',
            'format':   bold
        }
    )

    worksheet.conditional_format(f'B{start_table}:G{end_table}', 
        {
            'type': 'no_blanks',
            'format': table
        }
    )
    worksheet.conditional_format(f'B{start_table}:G{end_table}', 
        {
            'type': 'blanks',
            'format': table
        }
    )
    worksheet.conditional_format(f'B{start_table}:G{end_table}', {
                                        'type': 'cell',
                                        'criteria': '>',
                                        'value': 999,
                                        'format': num_format})


    worksheet.set_column(1, 2, 20)
    worksheet.set_column(3, 3, 15)
    worksheet.set_column(4, 4, 13)
    worksheet.set_column(5, 5, 20)

writer.save()
writer.close()

d:\Projects\Excel-to-SQL-snippets\env\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
